# Yoga Studio Corporate Raider

## Introduction / Business Problem

I have been approached by a self-described "Corporate Raider" who dreams of owning a chain of yoga studios.  Rather than build them from the ground up, he'd like to find a group of at least 5 yoga studios that he can purchase and put together into a chain.  The customer lives in Southern California and wants the yoga chain to be based in the Los Angeles area.  

The customer is willing to build his yoga empire inland if that is all he can find, but would prefer to have it closer to the coast.  The 5 yoga studios need to be relatively close to each other, preferably in the same neighborhood.  This customer is not concerned about price because he plans to make the current owners an offer they can't refuse.

## Data

The data that I will use to solve this problem will primarily come from the Foursquare API.  From Foursquare, I will need to filter results for venues in categories containing the word "Yoga" and pull the following information for each venue:

<ul>
<li>Name</li>
<li>Rating<li>
<li>Location (latitude / longitude)<li>
<li>Address<li>
</ul>

I will also pull in a zip code list of the LA area from this website:

http://www.laalmanac.com/communications/cm02_communities.php

This data will help determine which neighborhood each yoga studio is in.